In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy import stats
from apyori import apriori

In [2]:
pd.set_option('display.max_rows', 200)

In [3]:
df = pd.read_csv('data/mma_mart.csv')

### Seperate refrigerated beerages

In [4]:
df.loc[df['aisle'] == 'refrigerated', 'department'] = 'beverages_refrigerated'
df.loc[df['department'] == 'beverages', 'department'] = 'beverages_non_refrigerated'

### Remove 'missing' department and 'missing' aisle

In [5]:
df = df[df['department'] != 'missing']

### Mark products as refrigerated or frozen

In [6]:
df['frozen'] = df['department'] == 'frozen'
dept_refrigerated = ['dairy eggs', 'deli', 'meat seafood', 'beverages_refrigerated']
df['refrigerated'] = df['department'].isin(dept_refrigerated)

### Largest Remainder Method

In [7]:
def largest_remainder_method(decimal_series):
    total_percentage = 100
    rounded_percentages = [round(decimal * total_percentage) for decimal in decimal_series]

    # Calculate the total after rounding
    rounded_total = sum(rounded_percentages)

    # Calculate the remainder for each percentage
    remainders = [(i, rounded_percentage % 1) for i, rounded_percentage in enumerate(rounded_percentages)]

    # Sort the remainders in descending order
    remainders.sort(key=lambda x: x[1], reverse=True)

    # Distribute the remaining percentage to the largest remainders
    remaining_percentage = total_percentage - rounded_total
    for i in range(remaining_percentage):
        index = remainders[i % len(remainders)][0]
        rounded_percentages[index] += 1

    return rounded_percentages

### Create Refrigerated products dataframe

In [8]:
df_refrige = df[df['refrigerated'] == True]

In [9]:
refridge_dept_counts = df_refrige.groupby('department').agg(
    products_sold=pd.NamedAgg(column='product_name', aggfunc='nunique'),
    total_items_sold=pd.NamedAgg(column='product_name', aggfunc='size')
).reset_index()

refridge_dept_counts['store_util'] = refridge_dept_counts['products_sold'] / refridge_dept_counts['products_sold'].sum()
refridge_dept_counts['dept_dominance'] = refridge_dept_counts['total_items_sold'] / refridge_dept_counts['total_items_sold'].sum()
refridge_dept_counts['aisle_allocation'] = largest_remainder_method(((refridge_dept_counts['store_util']+refridge_dept_counts['dept_dominance'] ) / 2))
#refridge_dept_counts.loc[refridge_dept_counts['department'] == 'beverages', 'department'] = 'beverages_refrigerated'

In [10]:
refridge_dept_counts

,department,products_sold,total_items_sold,store_util,dept_dominance,aisle_allocation
0,beverages_refrigerated,544,17663,0.104797,0.074930,9
1,dairy eggs,2886,164468,0.555962,0.697705,63
2,deli,1069,32008,0.205933,0.135784,17
3,meat seafood,692,21588,0.133308,0.091581,11


In [11]:
# Create a dictionary from the two columns
dept_allocation = dict(zip(refridge_dept_counts['department'], refridge_dept_counts['aisle_allocation']))

### Create General products dataframe

In [12]:
df_general = df[(df['refrigerated'] == False) & (df['frozen'] == False)]

In [13]:
general_dept_counts = df_general.groupby('department').agg(
    products_sold=pd.NamedAgg(column='product_name', aggfunc='nunique'),
    total_items_sold=pd.NamedAgg(column='product_name', aggfunc='size')
).reset_index()

general_dept_counts['store_util'] = general_dept_counts['products_sold'] / general_dept_counts['products_sold'].sum()
general_dept_counts['dept_dominance'] = general_dept_counts['total_items_sold'] / general_dept_counts['total_items_sold'].sum()
general_dept_counts['aisle_allocation'] = largest_remainder_method((general_dept_counts['store_util']+general_dept_counts['dept_dominance'] ) / 2)
general_dept_counts['aisle_allocation'] = general_dept_counts['aisle_allocation'] * 8
#general_dept_counts.loc[general_dept_counts['department'] == 'beverages', 'department'] = 'beverages_non_refrigerated'

In [14]:
general_dept_counts

,department,products_sold,total_items_sold,store_util,dept_dominance,aisle_allocation
0,alcohol,606,4580,0.023100,0.006725,16
1,babies,799,12872,0.030457,0.018900,16
2,bakery,1204,35806,0.045895,0.052575,40
3,beverages_non_refrigerated,2709,64260,0.103263,0.094356,80
4,breakfast,905,21585,0.034497,0.031694,24
5,bulk,33,1087,0.001258,0.001596,0
6,canned goods,1557,32486,0.059350,0.047701,40
7,dry goods pasta,1366,26096,0.052070,0.038318,40
8,household,2028,22445,0.077304,0.032957,48
9,international,772,8248,0.029427,0.012111,16


In [15]:
# Create a new dictionary from the DataFrame
temp_dict = dict(zip(general_dept_counts['department'], general_dept_counts['aisle_allocation']))
# Add the new dictionary to the existing dictionary
dept_allocation.update(temp_dict)
dept_allocation['frozen'] = 100

In [16]:
dept_allocation

{'beverages_refrigerated': 9,
 'dairy eggs': 63,
 'deli': 17,
 'meat seafood': 11,
 'alcohol': 16,
 'babies': 16,
 'bakery': 40,
 'beverages_non_refrigerated': 80,
 'breakfast': 24,
 'bulk': 0,
 'canned goods': 40,
 'dry goods pasta': 40,
 'household': 48,
 'international': 16,
 'other': 8,
 'pantry': 88,
 'personal care': 64,
 'pets': 8,
 'produce': 192,
 'snacks': 120,
 'frozen': 100}

### Check correlation between store_util and dept_dominance


In [17]:
# Calculate the correlation coefficient between the two variables
corr_coef = department_counts['store_util'].corr(department_counts['dept_dominance'])

print(f'Correlation coefficient between store_util and dept_dominance: {corr_coef}')

NameError: name 'department_counts' is not defined

Weak correlation between the number of products within each department and the total number of sales between each department. This means having more products in your department does not lead to more department sales.

### Confidence Interval of total_quantity_sold for each department

In [ ]:
department_stats = {}

for department, data in top_1000.groupby('department'):
    mean_quantity = data['total_quantity_sold'].mean()
    std_quantity = data['total_quantity_sold'].std()
    sem_quantity = stats.sem(data['total_quantity_sold'])
    confidence_interval = [round(val, 2) for val in stats.t.interval(0.95, len(data)-1, loc=mean_quantity, scale=sem_quantity)]

    department_stats[department] = {
        'mean_quantity': round(mean_quantity, 2),
        'std_quantity': round(std_quantity, 2),
        'confidence_interval': confidence_interval
    }

# Display the results
for department, info in department_stats.items():
    print(f"Department: {department}")
    print(f"Mean Quantity Sold: {info['mean_quantity']}")
    print(f"Standard Deviation: {info['std_quantity']}")
    print(f"Confidence Interval: {info['confidence_interval']}")
    print()

### Top 1000 items sold

In [ ]:
# Group by 'product_name' and sum rows for sales count
df_quantity_sold = df.groupby('product_name').size().reset_index(name='total_quantity_sold')

# Merge with df to carryover product details
df_quantity_sold = pd.merge(df_quantity_sold, df.drop_duplicates('product_name'), on='product_name')
df_quantity_sold = df_quantity_sold.sort_values(by='total_quantity_sold', ascending=False)
# Find top 1000 products
top_1000 = df_quantity_sold.head(1000)

In [ ]:
top_1000[top_1000['department'] == 'beverages_refrigerated']

### Find department breakdown of top_1000 products

In [ ]:
top_1000_department_counts = top_1000.groupby('department').agg(
    products_sold=pd.NamedAgg(column='product_name', aggfunc='nunique'),
).reset_index()

In [ ]:
top_1000_department_counts['dept_top_1000_dominance'] = top_1000_department_counts['products_sold'] / top_1000_department_counts['products_sold'].sum()

In [ ]:
top_1000_department_counts

### Find refridgerated breakdown of top_1000 products

In [ ]:
top_1000_refrigerated = top_1000[top_1000['refrigerated'] == True]

print(f'Number of refrigerated products in the top_1000: {top_1000_refrigerated.shape[0]}')

In [ ]:
# Dateframe of the top 100 refrigerated products
top_100_refrigerated = top_1000_refrigerated.head(100)

### Picking Top 1000

In [ ]:
selected_rows = []

# Iterate through the dictionary and DataFrame to pick items
for department, num_items in dept_allocation.items():
    department_rows = df_quantity_sold[df_quantity_sold['department'] == department].head(num_items)
    selected_rows.append(department_rows)

# Concatenate the selected rows into a new DataFrame
picked_df = pd.concat(selected_rows)

In [ ]:
picked_df.head(10)

In [ ]:
#picked_df.to_csv('items_1000.csv', index=False)

In [ ]:
# Filter rows where 'column1' contains the word 'garlic'
filtered_df = picked_df[picked_df['product_name'].str.contains('apple', case=False)]

filtered_df